In [2]:
import os
import sys
import glob
import scipy
import skbio
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from collections import defaultdict
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline
sns.set_style('whitegrid')
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['pdf.fonttype'] = 42

from IPython.display import display, HTML
import tRep

import warnings
warnings.filterwarnings("ignore")

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 200

### Load read depth table

In [5]:
rfile = '/groups/banfield/projects/human/data3/clou/wastewater/Datasheets/waster_water_2020Aug20.readDepths'

table = defaultdict(list)
with open(rfile, 'r') as f:
    for line in f.readlines():
        words = line.strip().split()
        table['sample'].append(words[0])
        table['read_bases'].append(int(words[1]) * 2)
        table['reads'].append(int(words[2]) * 2)

Rdb = pd.DataFrame(table)

Rdb['read_length'] = [int(x)/int(y) for x,y in zip(Rdb['read_bases'], Rdb['reads'])]
Rdb['Illumina_enrichment'] = ['enriched' if 'p' in x else "unenriched" for x in Rdb['sample']]
Rdb = Rdb.sort_values('sample').reset_index(drop=True)
Rdb

,sample,read_bases,reads,read_length,Illumina_enrichment
0,5_13_A_S1,6107540,82174,74.324482,unenriched
1,5_13_C_S2,1796238,24144,74.396869,unenriched
2,5_19_A_S4,25431914,343252,74.091088,unenriched
3,5_19_F_S3,24851304,334748,74.238842,unenriched
4,5_28_A_S28,71023874,978422,72.590226,unenriched
5,5_28_E_S26,72838764,1009198,72.174899,unenriched
6,5_28_I_S25,89771642,1248062,71.928832,unenriched
7,630-N,4177966696,54973246,76.000000,unenriched
8,630-S1,2563889792,33735392,76.000000,unenriched
9,630-S2,4420453816,58163866,76.000000,unenriched


In [9]:
# fixing a naming type

Rdb.at[15,'sample'] = '6_09_S1_S33'
Rdb

,sample,read_bases,reads,read_length,Illumina_enrichment
0,5_13_A_S1,6107540,82174,74.324482,unenriched
1,5_13_C_S2,1796238,24144,74.396869,unenriched
2,5_19_A_S4,25431914,343252,74.091088,unenriched
3,5_19_F_S3,24851304,334748,74.238842,unenriched
4,5_28_A_S28,71023874,978422,72.590226,unenriched
5,5_28_E_S26,72838764,1009198,72.174899,unenriched
6,5_28_I_S25,89771642,1248062,71.928832,unenriched
7,630-N,4177966696,54973246,76.000000,unenriched
8,630-S1,2563889792,33735392,76.000000,unenriched
9,630-S2,4420453816,58163866,76.000000,unenriched


In [10]:
# save

Rdb.to_csv('/groups/banfield/projects/human/data3/clou/wastewater/Datasheets/waster_water_readDepths_formatted.csv',\
           index=False)